# Compare Topological Assignment Improved and version2

In [1]:
from TPImprovedV2 import TopologicalAssignmentImprovedv2
from TPImproved import TopologicalAssignmentImproved
from SwapModified import BasicSwapModified
from cost import *

from qiskit import QuantumCircuit
from qiskit.test.mock.backends import FakeBrooklyn
from qiskit.transpiler import Layout, CouplingMap

from qiskit.converters import circuit_to_dag
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import dag_to_circuit

import os
from tabulate import tabulate

In [2]:
# info device 65 qubits -- create DAG with 65 qubits
backend = FakeBrooklyn()
num_qubits =  backend.configuration().n_qubits
coupling_list = backend.configuration().coupling_map
coupling_map = CouplingMap(couplinglist=coupling_list)
device_qc = QuantumCircuit(num_qubits, num_qubits)
device_DAG = circuit_to_dag(device_qc)

In [5]:
def compare_cost(qc, n_most_used):
    # for topological assignment improved - take n most used qubit
    total_cost = []
    for n in range(n_most_used):
        qc_copy = qc.copy()
        new_input_DAG = circuit_to_dag(qc_copy) 

        # build the input_DAG on the device_DAG (65 qubits)
        new_device_qc = QuantumCircuit(num_qubits, num_qubits)
        new_device_DAG = circuit_to_dag(new_device_qc) 
        new_device_DAG.compose(new_input_DAG)

        tpi = TopologicalAssignmentImproved(coupling_map)
        tp_layouti = tpi.run(new_device_DAG, n + 1) # n is from 0 to (n_most_used - 1)
        basic_pass_modifiedi = BasicSwapModified(coupling_map, tp_layouti)
        new_dag_tp_assignmnet_i = basic_pass_modifiedi.run(new_device_DAG)
        cost_tp_assignment_i = compute_cost(new_dag_tp_assignmnet_i)
        total_cost.append(cost_tp_assignment_i)
        print('tp assignment improved - ',n + 1, ' most used qubit - ok \n')
        
        qc_copy2 = qc.copy()
        new_input_DAG2 = circuit_to_dag(qc_copy2) 

        # build the input_DAG on the device_DAG (65 qubits)
        new_device_qc2 = QuantumCircuit(num_qubits, num_qubits)
        new_device_DAG2 = circuit_to_dag(new_device_qc2) 
        new_device_DAG2.compose(new_input_DAG2)

        tpi2 = TopologicalAssignmentImprovedv2(coupling_map)
        tp_layouti2 = tpi2.run(new_device_DAG2, n + 1) # n is from 0 to (n_most_used - 1)
        basic_pass_modifiedi2 = BasicSwapModified(coupling_map, tp_layouti2)
        new_dag_tp_assignmnet_i2 = basic_pass_modifiedi2.run(new_device_DAG2)
        cost_tp_assignment_i2 = compute_cost(new_dag_tp_assignmnet_i2)
        total_cost.append(cost_tp_assignment_i2)
        print('tp assignment improved - v2 - ',n + 1, ' most used qubit - ok \n')
        
    return total_cost

In [8]:
folderpath = r".\..\original" # make sure to put the 'r' in front
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
table_rows = []

# for this project we just analyse up to 5 most used qubits
n_most_used = 5

#for i in range(5):
for i in range(len(filepaths)):
    print('------------iteration ', i, '-----------  file: ', filepaths[i], '----------')
    input_path = filepaths[i]
    qc = QuantumCircuit.from_qasm_file(path=input_path)
    costs = compare_cost(qc, n_most_used)
    print('tot------', costs)
    row = [input_path.rsplit('\\', 1)[1]]
    row.extend(costs)
    table_rows.append(row)  

------------iteration  0 -----------  file:  .\..\original\adder-13.qasm ----------
gates---
 {'rz': 534, 'rx': 0, 'sx': 26, 'cx': 494, 'swap': 668, 'barrier': 1, 'measure': 26}


Cost of the circuit: 25006
tp assignment improved -  1  most used qubit - ok 

gates---
 {'rz': 534, 'rx': 0, 'sx': 26, 'cx': 494, 'swap': 766, 'barrier': 1, 'measure': 26}


Cost of the circuit: 27946
tp assignment improved - v2 -  1  most used qubit - ok 

gates---
 {'rz': 534, 'rx': 0, 'sx': 26, 'cx': 494, 'swap': 676, 'barrier': 1, 'measure': 26}


Cost of the circuit: 25246
tp assignment improved -  2  most used qubit - ok 

gates---
 {'rz': 534, 'rx': 0, 'sx': 26, 'cx': 494, 'swap': 688, 'barrier': 1, 'measure': 26}


Cost of the circuit: 25606
tp assignment improved - v2 -  2  most used qubit - ok 

gates---
 {'rz': 534, 'rx': 0, 'sx': 26, 'cx': 494, 'swap': 708, 'barrier': 1, 'measure': 26}


Cost of the circuit: 26206
tp assignment improved -  3  most used qubit - ok 

gates---
 {'rz': 534, 'rx': 0, 

gates---
 {'rz': 2, 'rx': 0, 'sx': 1, 'cx': 15, 'swap': 0, 'measure': 16}


Cost of the circuit: 151
tp assignment improved -  3  most used qubit - ok 

gates---
 {'rz': 2, 'rx': 0, 'sx': 1, 'cx': 15, 'swap': 1, 'measure': 16}


Cost of the circuit: 181
tp assignment improved - v2 -  3  most used qubit - ok 

gates---
 {'rz': 2, 'rx': 0, 'sx': 1, 'cx': 15, 'swap': 1, 'measure': 16}


Cost of the circuit: 181
tp assignment improved -  4  most used qubit - ok 

gates---
 {'rz': 2, 'rx': 0, 'sx': 1, 'cx': 15, 'swap': 0, 'measure': 16}


Cost of the circuit: 151
tp assignment improved - v2 -  4  most used qubit - ok 

gates---
 {'rz': 2, 'rx': 0, 'sx': 1, 'cx': 15, 'swap': 1, 'measure': 16}


Cost of the circuit: 181
tp assignment improved -  5  most used qubit - ok 

gates---
 {'rz': 2, 'rx': 0, 'sx': 1, 'cx': 15, 'swap': 0, 'measure': 16}


Cost of the circuit: 151
tp assignment improved - v2 -  5  most used qubit - ok 

tot------ [151, 151, 151, 151, 151, 181, 181, 151, 181, 151]
------

gates---
 {'rz': 73, 'rx': 0, 'sx': 26, 'cx': 48, 'swap': 13, 'x': 4, 'measure': 3}


Cost of the circuit: 896
tp assignment improved - v2 -  2  most used qubit - ok 

gates---
 {'rz': 73, 'rx': 0, 'sx': 26, 'cx': 48, 'swap': 28, 'x': 4, 'measure': 3}


Cost of the circuit: 1346
tp assignment improved -  3  most used qubit - ok 

gates---
 {'rz': 73, 'rx': 0, 'sx': 26, 'cx': 48, 'swap': 28, 'x': 4, 'measure': 3}


Cost of the circuit: 1346
tp assignment improved - v2 -  3  most used qubit - ok 

gates---
 {'rz': 73, 'rx': 0, 'sx': 26, 'cx': 48, 'swap': 28, 'x': 4, 'measure': 3}


Cost of the circuit: 1346
tp assignment improved -  4  most used qubit - ok 

gates---
 {'rz': 73, 'rx': 0, 'sx': 26, 'cx': 48, 'swap': 28, 'x': 4, 'measure': 3}


Cost of the circuit: 1346
tp assignment improved - v2 -  4  most used qubit - ok 

gates---
 {'rz': 73, 'rx': 0, 'sx': 26, 'cx': 48, 'swap': 34, 'x': 4, 'measure': 3}


Cost of the circuit: 1526
tp assignment improved -  5  most used qubit - ok 

ga

gates---
 {'rz': 797, 'rx': 0, 'sx': 290, 'cx': 576, 'swap': 476, 'x': 16, 'measure': 7}


Cost of the circuit: 20330
tp assignment improved -  5  most used qubit - ok 

gates---
 {'rz': 797, 'rx': 0, 'sx': 290, 'cx': 576, 'swap': 476, 'x': 16, 'measure': 7}


Cost of the circuit: 20330
tp assignment improved - v2 -  5  most used qubit - ok 

tot------ [35090, 27620, 23900, 33890, 24920, 24830, 24950, 33920, 20330, 20330]
------------iteration  14 -----------  file:  .\..\original\multiplier-2.qasm ----------
gates---
 {'rz': 163, 'rx': 0, 'sx': 8, 'cx': 152, 'swap': 145, 'barrier': 1, 'measure': 8}


Cost of the circuit: 5878
tp assignment improved -  1  most used qubit - ok 

gates---
 {'rz': 163, 'rx': 0, 'sx': 8, 'cx': 152, 'swap': 150, 'barrier': 1, 'measure': 8}


Cost of the circuit: 6028
tp assignment improved - v2 -  1  most used qubit - ok 

gates---
 {'rz': 163, 'rx': 0, 'sx': 8, 'cx': 152, 'swap': 119, 'barrier': 1, 'measure': 8}


Cost of the circuit: 5098
tp assignment im

gates---
 {'rz': 640, 'rx': 0, 'sx': 27, 'cx': 612, 'swap': 1098, 'measure': 27}


Cost of the circuit: 39087
tp assignment improved - v2 -  1  most used qubit - ok 

gates---
 {'rz': 640, 'rx': 0, 'sx': 27, 'cx': 612, 'swap': 1373, 'measure': 27}


Cost of the circuit: 47337
tp assignment improved -  2  most used qubit - ok 

gates---
 {'rz': 640, 'rx': 0, 'sx': 27, 'cx': 612, 'swap': 1274, 'measure': 27}


Cost of the circuit: 44367
tp assignment improved - v2 -  2  most used qubit - ok 

gates---
 {'rz': 640, 'rx': 0, 'sx': 27, 'cx': 612, 'swap': 1448, 'measure': 27}


Cost of the circuit: 49587
tp assignment improved -  3  most used qubit - ok 

gates---
 {'rz': 640, 'rx': 0, 'sx': 27, 'cx': 612, 'swap': 1345, 'measure': 27}


Cost of the circuit: 46497
tp assignment improved - v2 -  3  most used qubit - ok 

gates---
 {'rz': 640, 'rx': 0, 'sx': 27, 'cx': 612, 'swap': 1393, 'measure': 27}


Cost of the circuit: 47937
tp assignment improved -  4  most used qubit - ok 

gates---
 {'r

gates---
 {'rz': 725, 'rx': 0, 'sx': 52, 'cx': 667, 'swap': 1300, 'x': 1, 'measure': 26}


Cost of the circuit: 45722
tp assignment improved -  5  most used qubit - ok 

gates---
 {'rz': 725, 'rx': 0, 'sx': 52, 'cx': 667, 'swap': 1239, 'x': 1, 'measure': 26}


Cost of the circuit: 43892
tp assignment improved - v2 -  5  most used qubit - ok 

tot------ [44462, 40472, 45542, 43382, 45152, 44312, 41102, 41132, 45722, 43892]
------------iteration  23 -----------  file:  .\..\original\qpe-4.qasm ----------
gates---
 {'rz': 37, 'rx': 0, 'sx': 8, 'cx': 26, 'swap': 9, 'x': 1, 'measure': 4}


Cost of the circuit: 538
tp assignment improved -  1  most used qubit - ok 

gates---
 {'rz': 37, 'rx': 0, 'sx': 8, 'cx': 26, 'swap': 14, 'x': 1, 'measure': 4}


Cost of the circuit: 688
tp assignment improved - v2 -  1  most used qubit - ok 

gates---
 {'rz': 37, 'rx': 0, 'sx': 8, 'cx': 26, 'swap': 8, 'x': 1, 'measure': 4}


Cost of the circuit: 508
tp assignment improved -  2  most used qubit - ok 

gat

IndexError: list index out of range

In [9]:
table = [['benchmark name', 'tpi_1', 'v2', 'tpi_2', 'v2', 'tpi_3', 'v2', 'tpi_4', 'v2', 'tpi_5', 'v2']]
table.extend(table_rows)

print('--------------------------- Cost in terms of gates-------------------------------------------------------')
print(tabulate(table,headers='firstrow', tablefmt='fancy_grid', numalign='center'))

--------------------------- Cost in terms of gates-------------------------------------------------------
╒═══════════════════╤═════════╤═══════╤═════════╤═══════╤═════════╤═══════╤═════════╤═══════╤═════════╤═══════╕
│ benchmark name    │  tpi_1  │  v2   │  tpi_2  │  v2   │  tpi_3  │  v2   │  tpi_4  │  v2   │  tpi_5  │  v2   │
╞═══════════════════╪═════════╪═══════╪═════════╪═══════╪═════════╪═══════╪═════════╪═══════╪═════════╪═══════╡
│ adder-13.qasm     │  25006  │ 27946 │  25246  │ 25606 │  26206  │ 27646 │  29056  │ 30046 │  31906  │ 27196 │
├───────────────────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┤
│ adder-16.qasm     │  51172  │ 51262 │  45982  │ 44482 │  47812  │ 51592 │  49732  │ 53932 │  57442  │ 46432 │
├───────────────────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┤
│ adder-3.qasm      │   576   │  546  │   576   │  546  │   576   │  636  │   576   │  636  │   546   │  606  

# Let's compare

Entire results have been saved in: ".\tpi-vs-tpiv2-compact.png"

In [30]:
# let's build a list with these elements:
# el0: number of tpi_1 costs > v2
# el1: number of tpi_1 cost < v2
# el2: number of tpi_1 cost = v2
# ...

# create list structure
comparison_list = []
for i in range(n_most_used):
    comparison_list.append([0, 0, 0])
    
for row in table_rows:
    for i in range(n_most_used):
        j = i * 2 + 1
        if row[j] > row[j+1]:
            comparison_list[i][0] += 1
        elif row[j] < row[j+1]:
            comparison_list[i][1] += 1
        else:
            comparison_list[i][2] += 1
# show results
for elem in range(n_most_used):
    print('\n\n --- tpi_', elem + 1, ' ---\n')
    print('n. cases where \t tpi cost > v2: \t', comparison_list[elem][0])
    print('n. cases where \t tpi cost < v2: \t', comparison_list[elem][1])
    print('n. cases where \t tpi cost = v2: \t', comparison_list[elem][2])



 --- tpi_ 1  ---

n. cases where 	 tpi cost > v2: 	 9
n. cases where 	 tpi cost < v2: 	 8
n. cases where 	 tpi cost = v2: 	 9


 --- tpi_ 2  ---

n. cases where 	 tpi cost > v2: 	 11
n. cases where 	 tpi cost < v2: 	 9
n. cases where 	 tpi cost = v2: 	 6


 --- tpi_ 3  ---

n. cases where 	 tpi cost > v2: 	 5
n. cases where 	 tpi cost < v2: 	 16
n. cases where 	 tpi cost = v2: 	 5


 --- tpi_ 4  ---

n. cases where 	 tpi cost > v2: 	 8
n. cases where 	 tpi cost < v2: 	 11
n. cases where 	 tpi cost = v2: 	 7


 --- tpi_ 5  ---

n. cases where 	 tpi cost > v2: 	 6
n. cases where 	 tpi cost < v2: 	 10
n. cases where 	 tpi cost = v2: 	 10


It seems that using **more n-most-used-qubits** the v2 of TopologicalAssignmentImproved v2 performs much better, while **for n=1 or n=2** the previous version performs slightly better.

The question would be: why?

# Conclusions

Comparing every mapper with each other it's obvious that **the best one is the one which uses less n-most-used-qubits**. This is probably because **the most-used-qubits don't have a lot of interactions between each others** and we are statically mapping them to physical qubits which are close to each other (for convenience...). 

An improvement would be to find how much far away from each other is the best way to map these n-most-used-qubits.

In [44]:
# try to define the best one overall
best_costs_count = []
for i in range(n_most_used * 2):
    best_costs_count.append(0)

for row in table_rows:
    best_cost = min(row[1:])
    best_index = row.index(best_cost)
    best_costs_count[best_index - 1] += 1

print('\n\nNumber of times when the best is: ')
for i in range(n_most_used):
    print('\ntpi_', i + 1, ':\t', best_costs_count[i * 2])
    print('\ntpi_', i + 1, '(v2):\t', best_costs_count[i * 2 + 1])



Number of times when the best is: 

tpi_ 1 :	 9

tpi_ 1 (v2):	 6

tpi_ 2 :	 4

tpi_ 2 (v2):	 2

tpi_ 3 :	 1

tpi_ 3 (v2):	 1

tpi_ 4 :	 1

tpi_ 4 (v2):	 1

tpi_ 5 :	 1

tpi_ 5 (v2):	 0
